# Sankey Diagram of HS Correlations

In [29]:
import pandas as pd
import plotly.graph_objects as go
import chart_studio.plotly as py
import plotly

In [30]:
#Processing HS Correlation Tables

HSTABLES = "https://unstats.un.org/unsd/trade/classifications/tables/CompleteCorrelationsOfHS-SITC-BEC_20170606.xlsx"
HS = ['HS92', 'HS96', 'HS02', 'HS07', 'HS12', 'HS17'] #should be updated if changed
data = pd.read_excel(HSTABLES,
                     usecols= HS,
                     dtype = {v:'object' for v in HS}).dropna().drop_duplicates()
for col in HS: 
    data[col] = data[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
data['count'] = 1

In [31]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464',"#002060"]
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
    
    sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 15
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [32]:
#Declare your position
position = input()
version = position[:4]

 HS12-010130


In [33]:
fig = genSankey(data[data[version]==position], cat_cols = HS, value_cols = 'count', title = "HS02-010110")
plotly.offline.plot(fig, validate=False)

'temp-plot.html'